# Задание

1. Используя Gym Atari, выберите любую игру с более чем 4-мя действиями.
2. Обучите игру на стратегиях случайного поиска и восхождения на вершину.
3. Создайте видео с наибольшим вознаграждением и вставьте его в колаб.
  

In [ ]:
#@title Установка зависимостей
!pip install "gymnasium[atari]"
!pip install autorom[accept-rom-license]
!pip install moviepy
!pip install numpy
!pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=autorom_accept_rom_license-0.6.1-py3-none-any.whl size=446709 sha256=26c61c7dac405f74e527a49f9633baaf373eb4283bb4ad6b343adf986dadec02
  Stored in directory: /root/.cache/pip/wheels/99/f1/ff/c6966c034a8259164bdc9deb4d1ea839f119474638100e6645
Successfully built AutoROM.accept-rom-license


In [ ]:
#@title Импорты
import gymnasium as gym
import numpy as np
import cv2
from moviepy.editor import ImageSequenceClip
import io
import base64
from IPython.display import HTML, display
import time
import os
import ale_py

gym.register_envs(ale_py)

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



In [ ]:
#@title Константы и параметры

GAME_NAME = "ALE/MsPacman-v5"
FULL_ACTION_SPACE = False

# Random Search
N_EPISODES_RS = 500

# Hill Climbing
N_EPISODES_HC = 500
NOISE_START = 0.5        # начальный шум
NOISE_DECAY = 0.99       # уменьшение шума при неудаче
MAX_NOISE = 1.0          # максимальный шум

MAX_STEPS = 2000         # длительность эпизода
STATE_SIZE = 32          # финальное изображение 32×32 → 1024 фичи

# определение числа действий
env_tmp = gym.make(GAME_NAME, full_action_space=FULL_ACTION_SPACE)
N_ACTIONS = env_tmp.action_space.n
env_tmp.close()

FLAT_SIZE = STATE_SIZE * STATE_SIZE      # 1024
N_WEIGHTS = FLAT_SIZE * N_ACTIONS        # 1024 × actions

print("Игра:", GAME_NAME)
print("Действий:", N_ACTIONS)
print("Размер входа:", FLAT_SIZE)
print("Длина вектора весов:", N_WEIGHTS)

Игра: ALE/MsPacman-v5
Действий: 9
Размер входа: 1024
Длина вектора весов: 9216


In [ ]:
#@title Препроцессинг и функция выбора действия

def preprocess(state, size=STATE_SIZE):
    """Grayscale → Resize → Normalize → Flatten."""
    gray = cv2.cvtColor(state, cv2.COLOR_RGB2GRAY)
    small = cv2.resize(gray, (size, size), interpolation=cv2.INTER_AREA)
    flat = small.flatten().astype(np.float32) / 255.0
    return flat

def get_action(state, weight):
    x = preprocess(state)
    W = weight.reshape(FLAT_SIZE, N_ACTIONS)
    scores = np.dot(x, W)
    return np.argmax(scores)

In [ ]:
#@title Запуск эпизода и функция видео

def run_episode(env, weight, max_steps=MAX_STEPS, record=False):
    state, _ = env.reset()
    total_reward = 0
    frames = []

    for t in range(max_steps):
        if record:
            frames.append(env.render())

        action = get_action(state, weight)
        state, reward, done, truncated, _ = env.step(action)

        total_reward += reward
        if done or truncated:
            break

    return total_reward, frames


def show_video(frames, filename="episode.mp4", fps=30):
    clip = ImageSequenceClip(frames, fps=fps)
    clip.write_videofile(filename, codec="libx264", audio_codec="aac", verbose=False)

    video_bytes = open(filename, "rb").read()
    encoded = base64.b64encode(video_bytes).decode()

    display(HTML(f"""
    <video width=600 controls>
        <source src="data:video/mp4;base64,{encoded}" type="video/mp4">
    </video>
    """))

In [ ]:
#@title Random Search

env_rs = gym.make(GAME_NAME, full_action_space=FULL_ACTION_SPACE)

best_reward_rs = -1e9
best_weight_rs = None
rewards_rs = []

print("Запуск Random Search...")

for ep in range(N_EPISODES_RS):
    w = np.random.randn(N_WEIGHTS)
    reward, _ = run_episode(env_rs, w)

    rewards_rs.append(reward)

    if reward > best_reward_rs:
        best_reward_rs = reward
        best_weight_rs = w.copy()
        print(f"[НОВЫЙ ЛУЧШИЙ] Эпизод {ep+1}/{N_EPISODES_RS}: результат = {reward:.1f}")

    print(f"Эпизод {ep+1}/{N_EPISODES_RS}:\tрезультат = {reward:.1f}\tлучший результат = {best_reward_rs:.1f}")

env_rs.close()
print("Random Search завершён.")

Запуск Random Search...
[НОВЫЙ ЛУЧШИЙ] Эпизод 1/500: результат = 70.0
Эпизод 1/500:	результат = 70.0	лучший результат = 70.0
[НОВЫЙ ЛУЧШИЙ] Эпизод 2/500: результат = 120.0
Эпизод 2/500:	результат = 120.0	лучший результат = 120.0
Эпизод 3/500:	результат = 70.0	лучший результат = 120.0
Эпизод 4/500:	результат = 90.0	лучший результат = 120.0
Эпизод 5/500:	результат = 60.0	лучший результат = 120.0
Эпизод 6/500:	результат = 60.0	лучший результат = 120.0
Эпизод 7/500:	результат = 70.0	лучший результат = 120.0
Эпизод 8/500:	результат = 70.0	лучший результат = 120.0
Эпизод 9/500:	результат = 90.0	лучший результат = 120.0
[НОВЫЙ ЛУЧШИЙ] Эпизод 10/500: результат = 610.0
Эпизод 10/500:	результат = 610.0	лучший результат = 610.0
Эпизод 11/500:	результат = 210.0	лучший результат = 610.0
Эпизод 12/500:	результат = 220.0	лучший результат = 610.0
Эпизод 13/500:	результат = 60.0	лучший результат = 610.0
Эпизод 14/500:	результат = 90.0	лучший результат = 610.0
Эпизод 15/500:	результат = 60.0	лучший резу

In [ ]:
#@title Видео результата Random Search

print("Лучший reward Random Search, достигнутый в тренировке:", best_reward_rs)

# Запуск лучшего веса несколько раз для видео, чтобы
# нивелировать стохастичность и показать максимальный результат.
N_EVAL_RUNS = 5
best_eval_reward_rs = -1e9
best_eval_frames_rs = []

env_render_rs = gym.make(GAME_NAME, full_action_space=FULL_ACTION_SPACE, render_mode="rgb_array")

print(f"Проверка лучшего веса на {N_EVAL_RUNS} эпизодах для записи лучшего видео Random Search...")

for i in range(N_EVAL_RUNS):
    # Используем best_weight_rs, найденный в Random Search
    reward, frames = run_episode(env_render_rs, best_weight_rs, record=True)
    print(f"Оценочный прогон {i+1}: reward = {reward:.1f}")

    if reward > best_eval_reward_rs:
        best_eval_reward_rs = reward
        best_eval_frames_rs = frames

env_render_rs.close()

print(f"Максимальный reward для видео Random Search: {best_eval_reward_rs:.1f}")

show_video(best_eval_frames_rs, "best_random_search.mp4")

Лучший reward Random Search, достигнутый в тренировке: 1730.0
Проверка лучшего веса на 5 эпизодах для записи лучшего видео Random Search...
Оценочный прогон 1: reward = 1740.0
Оценочный прогон 2: reward = 1800.0
Оценочный прогон 3: reward = 1770.0
Оценочный прогон 4: reward = 1800.0
Оценочный прогон 5: reward = 1710.0
Максимальный reward для видео Random Search: 1800.0
Moviepy - Building video best_random_search.mp4.
Moviepy - Writing video best_random_search.mp4



Moviepy - Done !
Moviepy - video ready best_random_search.mp4


In [ ]:
#@title Hill Climbing

env_hc = gym.make(GAME_NAME, full_action_space=FULL_ACTION_SPACE)

NOISE = NOISE_START
w_best = np.random.randn(N_WEIGHTS)

r_best, _ = run_episode(env_hc, w_best)

print("Старт Hill Climbing. Стартовое вознаграждение:", r_best)

rewards_hc = []

for ep in range(N_EPISODES_HC):

    w_try = w_best + NOISE * np.random.randn(N_WEIGHTS)
    reward, _ = run_episode(env_hc, w_try)
    rewards_hc.append(reward)

    if reward >= r_best:
        r_best = reward
        w_best = w_try.copy()
        NOISE = min(NOISE * 1.05, MAX_NOISE)
        print(f"[НОВЫЙ ЛУЧШИЙ] Эпизод {ep+1}/{N_EPISODES_HC}: результат = {reward:.1f}, шум = {NOISE:.3f}")
    else:
        NOISE *= NOISE_DECAY

    print(f"Эпизод {ep+1}:\tрезультат = {reward:.1f}\tшум = {NOISE:.3f}\t лучший результат = {r_best:.1f}")

env_hc.close()
print("Hill Climbing завершён. Лучший результат:", r_best)

Старт Hill Climbing. Стартовое вознаграждение: 270.0
[НОВЫЙ ЛУЧШИЙ] Эпизод 1/500: результат = 360.0, шум = 0.525
Эпизод 1:	результат = 360.0	шум = 0.525	 лучший результат = 360.0
Эпизод 2:	результат = 210.0	шум = 0.520	 лучший результат = 360.0
Эпизод 3:	результат = 210.0	шум = 0.515	 лучший результат = 360.0
Эпизод 4:	результат = 70.0	шум = 0.509	 лучший результат = 360.0
Эпизод 5:	результат = 140.0	шум = 0.504	 лучший результат = 360.0
Эпизод 6:	результат = 210.0	шум = 0.499	 лучший результат = 360.0
Эпизод 7:	результат = 70.0	шум = 0.494	 лучший результат = 360.0
Эпизод 8:	результат = 70.0	шум = 0.489	 лучший результат = 360.0
Эпизод 9:	результат = 70.0	шум = 0.484	 лучший результат = 360.0
Эпизод 10:	результат = 70.0	шум = 0.480	 лучший результат = 360.0
[НОВЫЙ ЛУЧШИЙ] Эпизод 11/500: результат = 380.0, шум = 0.504
Эпизод 11:	результат = 380.0	шум = 0.504	 лучший результат = 380.0
Эпизод 12:	результат = 320.0	шум = 0.499	 лучший результат = 380.0
Эпизод 13:	результат = 320.0	шум = 0

In [ ]:
#@title Видео результата Hill Climbing

print("Лучший reward Hill Climbing, достигнутый в тренировке:", r_best)

# Запуск лучшего веса несколько раз для видео, чтобы
# нивелировать стохастичность и показать максимальный результат.
N_EVAL_RUNS = 5
best_eval_reward = -1e9
best_eval_frames = []

env_render_hc = gym.make(GAME_NAME, full_action_space=FULL_ACTION_SPACE, render_mode="rgb_array")

print(f"Проверка лучшего веса на {N_EVAL_RUNS} эпизодах для записи лучшего видео...")

for i in range(N_EVAL_RUNS):
    reward, frames = run_episode(env_render_hc, w_best, record=True)
    print(f"Оценочный прогон {i+1}: reward = {reward:.1f}")

    if reward > best_eval_reward:
        best_eval_reward = reward
        best_eval_frames = frames

env_render_hc.close()

print(f"Максимальный результат для видео Hill Climbing: {best_eval_reward:.1f}")

show_video(best_eval_frames, "best_hill_climbing.mp4")

Лучший reward Hill Climbing, достигнутый в тренировке: 2100.0
Проверка лучшего веса на 5 эпизодах для записи лучшего видео...
Оценочный прогон 1: reward = 1900.0
Оценочный прогон 2: reward = 430.0
Оценочный прогон 3: reward = 1900.0
Оценочный прогон 4: reward = 1900.0
Оценочный прогон 5: reward = 390.0
Максимальный результат для видео Hill Climbing: 1900.0
Moviepy - Building video best_hill_climbing.mp4.
Moviepy - Writing video best_hill_climbing.mp4



Moviepy - Done !
Moviepy - video ready best_hill_climbing.mp4


# Вывод по проделанной работе

В рамках данного проекта мной была реализована система обучения агента для игры MsPacman-v5 из среды Gym Atari, обладающей сложным дискретным пространством из девяти действий. Я использовал простую, но вычислительно эффективную линейную политику, где действие выбирается на основе взвешенной суммы пикселей препроцессированного изображения. Для сокращения размерности входных данных и повышения скорости обучения мной был применен агрессивный препроцессинг, включающий перевод кадров в оттенки серого и уменьшение их размера до 32x32 пикселей.

Мной были обучены две модели с использованием стратегий случайного поиска (Random Search, RS) и восхождения на вершину (Hill Climbing, HC). Ключевой оптимизацией для получения достоверных видеорезультатов, учитывая высокую стохастичность среды Atari, стало внедрение множественной оценки: лучший набор весов проверялся на пяти независимых эпизодах.

Сравнивая результаты, я обнаружил, что Random Search достиг максимального результата во время тренировки в 1730 очков, а при последующей оценке смог показать 1800 очков. Однако Hill Climbing, благодаря итеративному и направленному изменению весов, оказался более успешным: мной было зафиксировано вознаграждение в 2100 очков на этапе тренировки. Максимальный результат при оценке видео для Hill Climbing составил 1900 очков.

Эти данные убедительно демонстрируют, что, несмотря на простоту линейной политики, стратегия Hill Climbing, превзошла Random Search в способности находить более оптимальные локальные решения. Разница между лучшим тренировочным результатом (2100) и лучшим оценочным (1900) в Hill Climbing подчеркивает, насколько сильно стохастичность среды влияет на реальную производительность агента в каждом отдельном эпизоде. В целом, мною было подтверждено, что даже для низкоразмерных линейных политик направленный поиск обеспечивает лучшее качество, чем чистая случайность.